In [ ]:
import pandas as pd
import numpy as np



In [ ]:
bank_df = pd.read_excel("bank_deposit.xlsx")
bank_df

In [ ]:
# This line is to check NAN values in the DataFrame.
bank_df.isna().sum()
# This line is to check NAN values in the "contact" column.
#bank_df[bank_df["contact"].isna()]


In [ ]:
# Dropped the rows that contained NAN in the contact column.
bank_df= bank_df[bank_df['contact'].notna()]
bank_df

In [ ]:
def quantile_25(bank_df:pd.Series):
    return bank_df.quantile(.25)

def quantile_75(bank_df:pd.Series):
    return bank_df.quantile(.75)

In [ ]:
bank_df.pivot_table(values='balance', index=['Bank deposit(target)','loan'], aggfunc=[ quantile_25, np.median, quantile_75,np.max])

Function to update missing values for month, duration, campaign and p-days, based on median

In [ ]:
bank_df.head()

In [ ]:
def replace_w_median(bank_df):
    # find median for columns duration, campaign, pdays, and replace with mode for month
    bank_df['duration'].fillna(bank_df['duration'].median(), inplace=True)
    bank_df['campaign'].fillna(bank_df['campaign'].median(), inplace=True)
    bank_df['pdays'].fillna(bank_df['pdays'].median(), inplace=True)
    bank_df['month'] = bank_df['month'].fillna(bank_df['month'].mode()[0])

In [ ]:
replace_w_median(bank_df)

In [ ]:
bank_df[bank_df.isna().any(axis=1)]

In [ ]:
bank_df.isna().sum()

In [ ]:
bank_df.describe()